In [1]:

from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.common_tools.duckduckgo import duckduckgo_search_tool
from pydantic_ai.providers.ollama import OllamaProvider

import nest_asyncio

nest_asyncio.apply()

In [2]:
from src.models.agent_models import AuthorInfo

In [3]:
provider_url = "http://localhost:11434/v1"
model_name = "qwen3:4b"

provider = OllamaProvider(base_url=provider_url)
model = OpenAIChatModel(
    model_name=model_name,
    provider=provider,
)

In [4]:
author_info_prompt = """
You are an assistant that retrieves and summarizes book authors informations.
The names you're going to receive are ONLY book authors.

Instructions:

1. Search DuckDuckGo for the given buthor.ook title.
2. Gather accurate information from reliable sources (mainly wikipedia, publishers, reviews, and literary databases).
3. Write a concise, factual summary of the author (2–5 sentences) describing its life and importance.
4. EXTREMELY IMPORTANT: Output only the summary text — **do not include any introductions, reasoning, or phrases like “Based on the search results.”**
5. The output will be stored directly in a database.
"""

In [5]:

author_info_agent: Agent[None, AuthorInfo] = Agent(
    model=model,
    tools=[duckduckgo_search_tool()],
    system_prompt=author_info_prompt,
    output_type=AuthorInfo,
    retries=3
)

In [6]:
author_name = "Art Wolfe"
output_author = author_info_agent.run_sync(user_prompt=author_name)

In [10]:
output_author.output.model_dump_json(indent=4)

'{\n    "name": "Art Wolfe",\n    "birth_date": "1951-01-01",\n    "death_date": null,\n    "nationality": "American",\n    "biography": "Art Wolfe (born 1951) is an American photographer and conservationist renowned for his color images documenting landscapes, wildlife, and indigenous cultures across every continent. Over the past three decades, his work has been published in more than sixty books and has significantly contributed to environmental advocacy by highlighting Earth\'s ecosystems. His photography, praised for its ecological urgency and artistic precision, has earned recognition from organizations like the Wildlife Conservation Society and influenced public awareness through media platforms including television.",\n    "sex": "M",\n    "url": null\n}'